# Idea Generation & EDA
__Goals:__
1. Customer perspective: Build __Recommendation Engine__ to recommend similar homes in the same city when a user searches on one home 
    - Potentially recommend homes in a different city based on what user searched on / user history if I can find user history
    - Use image recognition for photos classification (what type of home is it)
    - Classification on __
    - Clustering for types of homes
     
2. Supplier perspective: __Sentiment__ score time-series for Hosts
    - Review Summaries over X period for hosts on what they can improve on and what they do well in
    - Linear Models for pricing (above or below average)
    - NLP and sentiment analysis on reviews for sentiment score time-series
    
3. Interactive Map and __Visualization__

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np
import math
import os
import pickle

In [7]:
os.chdir('/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/data')

In [8]:
listings_sf=pd.read_csv('SF/201808/listings_201808.csv')
print(listings_sf.shape)
listings_sf.head(3)

(6633, 96)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,20180806171132,2018-08-06,"Bright, Modern Garden Unit - 1BR/1B",Our bright garden unit overlooks a grassy back...,"Newly remodeled, modern, and bright garden uni...",Our bright garden unit overlooks a grassy back...,none,*Quiet cul de sac in friendly neighborhood *St...,...,t,STR-0001256,"{""SAN FRANCISCO""}",f,f,moderate,f,f,1,1.38
1,5858,https://www.airbnb.com/rooms/5858,20180806171132,2018-08-06,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,none,I love how our neighborhood feels quiet but is...,...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,1,0.99
2,7918,https://www.airbnb.com/rooms/7918,20180806171132,2018-08-06,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,Room rental-sunny view room/sink/Wi Fi (inner ...,Nice and good public transportation. 7 minute...,none,"Shopping old town, restaurants, McDonald, Whol...",...,t,NaN,"{""SAN FRANCISCO""}",f,f,strict_14_with_grace_period,f,f,9,0.16


In [9]:
listings_nyc=pd.read_csv('NYC/201808/listings_201808.csv')
print(listings_nyc.shape)
listings_nyc.head(2)

(50914, 96)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2515,https://www.airbnb.com/rooms/2515,20180806171147,2018-08-07,Stay at Chez Chic budget room #1,Step into our artistic spacious apartment and ...,-PLEASE BOOK DIRECTLY. NO NEED TO SEND A REQUE...,Step into our artistic spacious apartment and ...,none,NaN,...,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,3,1.42
1,2539,https://www.airbnb.com/rooms/2539,20180806171147,2018-08-07,Clean & quiet apt home by the park,Renovated apt home in elevator building.,"Spacious, renovated, and clean apt home, one b...",Renovated apt home in elevator building. Spaci...,none,Close to Prospect Park and Historic Ditmas Park,...,f,NaN,NaN,f,f,moderate,f,f,9,0.25


In [10]:
reviews_sf=pd.read_csv('SF/201808/reviews_201808.csv')
print(reviews_sf.shape)
reviews_sf.head(3)

(278884, 6)


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...


In [12]:
reviews_nyc=pd.read_csv('NYC/201808/reviews_201808.csv')
print(reviews_nyc.shape)
reviews_nyc.head(3)

(1009539, 6)


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2515,198,2008-10-13,2603,Jenny,Stephanie was a wonderful host! Her apartment ...
1,2515,859,2009-03-08,8455,Roland,Such a wonderful place and very close to the m...
2,2515,1083,2009-03-25,9759,Cem,I just got back from a trip to NYC during whic...


In [17]:
len(reviews_sf[reviews_sf['listing_id']==958])

152

In [16]:
len(reviews_sf.groupby('listing_id').count())

5491

In [20]:
reviews_sf.groupby('listing_id').count().sort_values(by=['id'], ascending=False)[:5]

,id,date,reviewer_id,reviewer_name,comments
listing_id,,,,,
205842,610,610,610,610,610
545685,596,596,596,596,596
8739,594,594,594,594,593
33577,579,579,579,579,579
748256,547,547,547,547,547


In [29]:
print(sorted(reviews_sf['date'], reverse=True)[:5])
print(sorted(reviews_sf['date'], reverse=False)[:5])

['2018-08-06', '2018-08-06', '2018-08-05', '2018-08-05', '2018-08-05']
['2009-05-03', '2009-05-23', '2009-06-12', '2009-07-15', '2009-07-23']


In [36]:
reviews_sf['comments'][reviews_sf['listing_id']==958][0]

"Our experience was, without a doubt, a five star experience. Holly and her husband, David, were the consummate hosts; friendly and accomodating while still honoring our privacy. The apartment was a charming layout with a full view and access to the home's garden The location is perfect for full engagement with the city; close to mass transit with walking proximity to the Haight, the Mission, the Castro and Golden Gate Park. I can't wait for our next visit.\r\n\r\nTed and Karen Wingerd"